In [1]:
from ast import literal_eval
from collections import deque
import time

import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds

Part 2 only solvable by Gemini 3 - also in the iteration with Gemini, I decided to change my initial solution for part 1 from a DFS to a BFS, which I could have done by myself for cleaning.

Anyway, part 2 is tricky, since I couldn't solve the problem with a dfs. I haven't used a lienar solver for a long time, so I needed the help of Gemini 3. (Still insisting to use scipy). 
This is my learning experience and hopefully I can remember this kind of problem the next time. So, I can start from this solution and adapt it to a similiar problem. Also, that's why I left the comments from Gemini in the script.

Addon - since I want to learn, I used Gemini3 for finding a solution only with numpy. This is part 3. This runs in < 2 minutes and uses the Branch and Bound algorithm with some optimizations.

## Day 10

### Test

In [2]:
test = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
"""

test = test.split("\n")[:-1]

In [3]:
def parse_instructions(instructions):

    lights = []
    buttons = []
    joltages = []
    
    for i in instructions:
        light, rest = i.split("] ")
        light = light[1:]
        light = light.replace(".", "0")
        light = light.replace("#", "1")
        lights.append(light)
    
        button, joltage = rest.split(" {")
    
        button = button.split()
        button = [literal_eval(i) for i in button]
        button = [i if isinstance(i,tuple) else tuple([i]) for i in button]
        buttons.append(button)
        
        joltage = joltage[:-1]
        joltage =joltage.split(",")
        joltage = [int(i) for i in joltage]
        joltages.append(joltage)

    return lights, buttons, joltages

In [4]:
def update_current(current, button):

    current_lst = list(current)
    for b in button:
        current_lst[b] = "1" if current_lst[b] == "0" else "0"
    new = "".join(current_lst)
    
    return new
    
    
def shortest_combination(buttons, target):

    start = "0" * len(target)
    
    queue = deque([(start, [])])
    
    visited = set()
    visited.add(start)
    

    while queue:
        current, path = queue.popleft()

        if current == target:
            return path

        if len(path) >= len(target)*2:
            continue

        for i, button in enumerate(buttons):
            
            new = update_current(current, button)

            if new not in visited:
                visited.add(new)

                queue.append((new, path + [i]))

    return None

In [5]:
results = []
lights, buttons, joltages = parse_instructions(test)

for i, j in zip(buttons, lights):

    result = shortest_combination(i, j)

    results.append(len(result))

In [6]:
assert sum(results) == 7

## Part 1

In [7]:
with open("../../../advent_of_code_input/2025/day_10/input.txt", "r") as f:
    data = f.read()

data = data.split("\n")[:-1]

In [8]:
start_time = time.perf_counter()

results = []

lights, buttons, joltages = parse_instructions(data)

for i, j in zip(buttons, lights):

    result = shortest_combination(i, j)

    results.append(len(result))
    
end_time = time.perf_counter()

elapsed = end_time - start_time
print(elapsed, "seconds")

0.08527958299964666 seconds


In [9]:
sum(results)

455

## Part 2

### Test

In [10]:
test = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
"""

test = test.split("\n")[:-1]

In [11]:
def solve_linear(buttons, joltage):
    
    num_vars = len(buttons)      # We want to find x0, x1, x2... (count for each button)
    num_equations = len(joltage)       # One equation per Joltage index

    # 2. BUILD THE MATRIX (A)
    A = np.zeros((num_equations, num_vars))
    
    for col, button in enumerate(buttons):
        for row in button:
            A[row, col] = 1

    # 3. CONSTRAINTS
    # In scipy, we set lower_bound = upper_bound = Target
    constraints = LinearConstraint(A, lb=joltage, ub=joltage)

        # By setting lb and ub to the same list (joltage):
        # 1. The result must be >= joltage
        # 2. The result must be <= joltage
    
    # 4. VARIABLES SETUP
    # Variables must be Integers (1)
    integrality = np.ones(num_vars) 

        # In the scipy.optimize.milp function, the integrality array tells the solver which variables must be whole numbers and which are allowed to be decimals (fractions).
        # 0 = Continuous (Decimals allowed, e.g., 2.5, 0.333)
        # 1 = Integer (Whole numbers only, e.g., 2.0, 0.0, 5.0)
    
    # Variables must be >= 0 (Cannot press a button -1 times)
    bounds = Bounds(lb=0, ub=np.inf)
    
    # 5. OBJECTIVE
    # We want to minimize the TOTAL number of presses.
    # Cost vector c = [1, 1, 1...] (all buttons cost 1 press)
    c = np.ones(num_vars)

        # What would happen if you changed it?
        # If you passed c = [0, 0, 0...] (Zeros):
        # The solver would find any valid solution that satisfies the math. It might return a solution with 1,000 steps because it "costs" nothing to add more steps. It wouldn't guarantee the shortest path.
        # If you passed c = [100, 1, 1...]:
        # You would be telling the solver: "Button 0 costs $100 to press, but others cost $1." The solver would try desperately to avoid using Button 0, even if it meant pressing Button 1 fifty times to compensate.
    
    # 6. SOLVE
    res = milp(c=c, constraints=constraints, integrality=integrality, bounds=bounds)

    if res.success:
        # Round to nearest int because solvers use floats internally
        counts = [int(round(x)) for x in res.x]
        
        path = []
        for i, count in enumerate(counts):
            path.extend([i] * count)
        return path
    else:
        return None


In [12]:
start_time = time.perf_counter()

results = []
lights, buttons, joltages = parse_instructions(test)

for n, (i, _, k) in enumerate(zip(buttons, lights, joltages)):

    result = solve_linear(i, k)

    results.append(len(result))

end_time = time.perf_counter()

elapsed = end_time - start_time
print(elapsed, "seconds")

0.0025512492284178734 seconds


In [13]:
assert sum(results) == 33

## Puzzle

In [14]:
with open("../../../advent_of_code_input/2025/day_10/input.txt", "r") as f:
    data = f.read()

data = data.split("\n")[:-1]

In [15]:
start_time = time.perf_counter()

results = []
lights, buttons, joltages = parse_instructions(data)

for n, (i, _, k) in enumerate(zip(buttons, lights, joltages)):

    result = solve_linear(i, k)

    results.append(len(result))

end_time = time.perf_counter()

elapsed = end_time - start_time
print(elapsed, "seconds")

0.07392475008964539 seconds


In [16]:
sum(results)

16978

## Gemini

General solution - Branch and Bound algorithm is applied on top of it

In [17]:
# # ==========================================
# # 1. THE SUBTRACTION TRICK (Helper Function)
# # ==========================================
# def simplify_rows(A, y):
#     """
#     Looks for "receipts" (rows) that are subsets of other receipts.
    
#     Analogy:
#     - Receipt 1: Apple + Banana = $10
#     - Receipt 2: Apple + Banana + Carrot = $15
    
#     Logic:
#     Since Receipt 1 is inside Receipt 2, the difference must be the Carrot!
#     $15 - $10 = $5. So, Carrot = $5.
    
#     We subtract the smaller row from the larger row to make the math simpler.
#     """
#     m, n = A.shape
    
#     # Only look at rows that still have items to solve (sum > 0)
#     active_rows = [i for i in range(m) if A[i].sum() > 0]
    
#     # Compare every active row against every other active row
#     for i in active_rows:
#         for j in active_rows:
#             if i == j: continue # Don't compare a row to itself
            
#             # MATH LOGIC: Check if Row i is completely inside Row j.
#             # We use bitwise AND (&). If (A & B) looks exactly like A, then A is inside B.
#             if np.array_equal(A[i] & A[j], A[i]):
                
#                 # Perform the subtraction: Row j becomes (Row j - Row i)
#                 A[j] -= A[i]
#                 y[j] -= y[i]
                
#                 # SAFETY CHECK: If a price becomes negative, this path is impossible.
#                 if y[j] < 0: return False 
                
#     return True # Simplification successful


# # ==========================================
# # 2. THE DETERMINISTIC SOLVER (The "Easy Wins")
# # ==========================================
# def solve_fast_deterministic(A, y, current_sol):
#     """
#     This function looks for obvious answers and keeps going until it gets stuck.
    
#     It looks for rows with exactly ONE variable, like "1 Apple = $5".
#     If it finds one, it writes down the answer and updates all other receipts.
#     """
#     while True:
#         # Step A: Run the Subtraction Trick from above
#         if not simplify_rows(A, y):
#             return None # We found a contradiction (like price < 0), so stop.

#         # Step B: Count how many unknowns are left in each row
#         row_counts = A.sum(axis=1)
        
#         # LOGIC CHECK: Are there any rows with 0 items but a price > 0?
#         # Example: "0 items = $10". This is impossible.
#         if np.any((row_counts == 0) & (y != 0)):
#             return None 

#         # Step C: Find rows with exactly 1 item left (The "Easy Wins")
#         solvable_rows = np.where(row_counts == 1)[0]
        
#         # If no easy wins are left, we stop and ask for a guess (return to main loop).
#         if len(solvable_rows) == 0:
#             break 

#         progress = False
        
#         for r_idx in solvable_rows:
#             # Double check: ensure the loop didn't already solve this row
#             if A[r_idx].sum() != 1: continue 
            
#             # Find WHICH item it is (get the index where the value is 1)
#             var_idx = np.argmax(A[r_idx]) 
#             val = y[r_idx] # The value (e.g., $5)
            
#             if val < 0: return None # Impossible negative price
            
#             # If we already knew this item's price, check if it matches.
#             # If we thought Apple=$5, but this row says Apple=$6, that's a conflict.
#             if current_sol[var_idx] is not None:
#                 if current_sol[var_idx] != val: return None
#             else:
#                 # Write down the solution!
#                 current_sol[var_idx] = int(val)
            
#             progress = True
            
#             # Step D: UPDATE EVERYTHING
#             # Now that we know Apple=$5, go to every other receipt and subtract $5 
#             # for every Apple found. Then cross "Apple" off the list (set column to 0).
#             y -= A[:, var_idx] * int(val)
#             A[:, var_idx] = 0 

#         # If we went through all rows and didn't find anything new, stop.
#         if not progress:
#             break
            
#     return current_sol


# # ==========================================
# # 3. THE RECURSIVE SOLVER (The "Smart Guesser")
# # ==========================================
# def recursive_solver_optimized(A, y, current_sol):
#     """
#     This is the 'Brain'. It tries to solve easily. If it gets stuck, 
#     it makes a smart guess and creates a parallel universe (recursion) 
#     to see if that guess works.
#     """
#     # 1. Try to simplify the system with logic first
#     res = solve_fast_deterministic(A, y, current_sol)
#     if res is None: return [] # This path was impossible (dead end)

#     # 2. Victory Check: If we have found all numbers (no None left), we win!
#     if None not in current_sol:
#         return [list(current_sol)]

#     # 3. HEURISTIC: Make a Smart Guess
#     # We need to pick a variable to guess (0, 1, 2...). 
#     # We shouldn't pick randomly. We want the smallest range of possibilities.
    
#     row_sums = A.sum(axis=1)
#     active_row_indices = np.where(row_sums > 0)[0]
    
#     if len(active_row_indices) == 0:
#         # Fallback: variables are unconstrained (rare), pick the first one.
#         best_var = current_sol.index(None)
#         limit = 2 
#     else:
#         # Filter for valid rows
#         valid_rows = active_row_indices[y[active_row_indices] >= 0]
#         if len(valid_rows) == 0: return []

#         # STRATEGY 1: Pick the receipt with the SMALLEST total price (y).
#         # Why? If "Apple + Pear = $2", the Apple can only be 0, 1, or 2. (3 choices).
#         # If "Apple + Pear = $100", the Apple could be 0..100. (101 choices).
#         # We want fewer choices to speed up the code!
#         best_row_idx = valid_rows[np.argmin(y[valid_rows])]
        
#         # STRATEGY 2: Within that receipt, pick the item that affects the most OTHER receipts.
#         # This helps "clean up" the matrix faster in the next step.
#         candidate_vars = np.where(A[best_row_idx] == 1)[0]
#         col_sums = A.sum(axis=0) # Count how often each variable appears
#         best_var = candidate_vars[np.argmax(col_sums[candidate_vars])]
        
#         # The limit for our guess is the total price + 1.
#         limit = y[best_row_idx] + 1

#     valid_results = []
    
#     # 4. The Loop: Try every possible integer for this variable
#     for k in range(limit):
#         # Create a "Save State" (Copy the data) so we don't mess up other guesses
#         A_next = A.copy()
#         y_next = y.copy()
#         sol_next = current_sol[:]
        
#         # Apply the Guess: Assume variable = k
#         sol_next[best_var] = k
        
#         # Update the math: subtract (1 * k) from the totals
#         y_next -= A_next[:, best_var] * k
#         A_next[:, best_var] = 0 # Remove variable
        
#         # RECURSE: Jump into the "universe" where this guess is true
#         valid_results.extend(recursive_solver_optimized(A_next, y_next, sol_next))
        
#         # Speed Tip: If you only need ONE solution, uncomment the next line:
#         # if len(valid_results) > 0: return valid_results

#     return valid_results


# # ==========================================
# # 4. MAIN SETUP
# # ==========================================
# def solve_system(buttons, joltage):
#     num_vars = len(buttons)
#     num_eqs = len(joltage)
    
#     # Create the Matrix (The Grid)
#     # We use 'int' (standard 64-bit integer) to prevent overflow errors on large numbers.
#     A = np.zeros((num_eqs, num_vars), dtype=int)
    
#     # Fill in the 1s where a button affects a joltage
#     for col, btn_indices in enumerate(buttons):
#         for row in btn_indices:
#             A[row, col] = 1
            
#     y = np.array(joltage, dtype=int)
    
#     # Start the Recursive Solver
#     # We start with a solution list full of 'None'
#     return recursive_solver_optimized(A, y, [None] * num_vars)


In [18]:
# start_time = time.perf_counter()

# results = []
# lights, buttons, joltages = parse_instructions(test)

# for n, (i, _, k) in enumerate(zip(buttons, lights, joltages)):

#     _results = solve_system(i, k)
#     result = min(_results,key=sum)

#     results.append(sum(result))

# end_time = time.perf_counter()

# elapsed = end_time - start_time
# print(elapsed, "seconds")

In [19]:
# assert sum(results) == 33

In [20]:
# start_time = time.perf_counter()

# results = []
# lights, buttons, joltages = parse_instructions(data)

# for n, (i, _, k) in enumerate(zip(buttons, lights, joltages)):

#     _results = solve_system(i, k)
#     result = min(_results,key=sum)

#     results.append(sum(result))

# end_time = time.perf_counter()

# elapsed = end_time - start_time
# print(elapsed, "seconds")

In [21]:
# sum(results)

In [22]:
# ==========================================
# 1. SUBTRACTION TRICK (Unchanged)
# ==========================================
def simplify_rows(A, y):
    m, n = A.shape
    active_rows = [i for i in range(m) if A[i].sum() > 0]
    for i in active_rows:
        for j in active_rows:
            if i == j: continue
            if np.array_equal(A[i] & A[j], A[i]):
                A[j] -= A[i]
                y[j] -= y[i]
                if y[j] < 0: return False 
    return True

# ==========================================
# 2. DETERMINISTIC SOLVER (Unchanged)
# ==========================================
def solve_fast_deterministic(A, y, current_sol):
    while True:
        if not simplify_rows(A, y): return None

        row_counts = A.sum(axis=1)
        if np.any((row_counts == 0) & (y != 0)): return None 

        solvable_rows = np.where(row_counts == 1)[0]
        if len(solvable_rows) == 0: break 

        progress = False
        for r_idx in solvable_rows:
            if A[r_idx].sum() != 1: continue 
            var_idx = np.argmax(A[r_idx]) 
            val = y[r_idx] 
            if val < 0: return None 
            
            if current_sol[var_idx] is not None:
                if current_sol[var_idx] != val: return None
            else:
                current_sol[var_idx] = int(val)
            progress = True
            y -= A[:, var_idx] * int(val)
            A[:, var_idx] = 0 
        if not progress: break
    return current_sol

# ==========================================
# 3. BRANCH AND BOUND SOLVER (Optimized)
# ==========================================
def recursive_solver_min_sum(A, y, current_sol, tracker):
    """
    Finds the solution with the MINIMAL sum.
    tracker: A dictionary {'min_sum': infinity, 'best_sol': None}
    """
    
    # 1. Calculate Current Cost
    # Sum up the variables we have found so far
    current_cost = sum(x for x in current_sol if x is not None)
    
    # OPTIMIZATION: "Pruning" (The Price Limit)
    # If what we have so far is ALREADY worse than our best record, stop.
    if current_cost >= tracker['min_sum']:
        return

    # OPTIMIZATION: "Look Ahead"
    # We look at the remaining 'y' (receipt totals).
    # Since all coefficients are 1, we know we need to buy at least enough 
    # items to satisfy the largest remaining receipt.
    # If (Current Cost + Estimated Future Cost) >= Best Record, stop.
    max_remaining_y = np.max(y) if len(y) > 0 else 0
    if current_cost + max_remaining_y >= tracker['min_sum']:
        return

    # 2. Run Deterministic Logic
    # Note: We pass copies to solve_fast_deterministic so we don't break the backtracking
    # But since solve_fast modifies in place, we rely on the copies made BEFORE calling this function
    # or inside the loop below. 
    res = solve_fast_deterministic(A, y, current_sol)
    if res is None: return 

    # 3. Check if Solved
    if None not in current_sol:
        total_sum = sum(current_sol)
        # If this is a new record, save it!
        if total_sum < tracker['min_sum']:
            tracker['min_sum'] = total_sum
            tracker['best_sol'] = list(current_sol)
            # print(f"New Record Found: {total_sum}") # Optional: to see progress
        return

    # 4. Smart Guessing Setup
    row_sums = A.sum(axis=1)
    active_row_indices = np.where(row_sums > 0)[0]
    
    if len(active_row_indices) == 0:
        best_var = current_sol.index(None)
        limit = 2 # Arbitrary small limit for unconstrained vars
    else:
        valid_rows = active_row_indices[y[active_row_indices] >= 0]
        if len(valid_rows) == 0: return 

        # Pick row with smallest total (tightest constraint)
        best_row_idx = valid_rows[np.argmin(y[valid_rows])]
        
        # Pick variable affecting most other rows
        candidate_vars = np.where(A[best_row_idx] == 1)[0]
        col_sums = A.sum(axis=0)
        best_var = candidate_vars[np.argmax(col_sums[candidate_vars])]
        
        limit = y[best_row_idx] + 1

    # 5. Recursive Loop
    for k in range(limit):
        
        # Another Pruning Check before diving in
        if (current_cost + k) >= tracker['min_sum']:
            break # Since k only gets bigger, we can stop the loop entirely here!

        A_next = A.copy()
        y_next = y.copy()
        sol_next = current_sol[:]
        
        sol_next[best_var] = k
        y_next -= A_next[:, best_var] * k
        A_next[:, best_var] = 0
        
        recursive_solver_min_sum(A_next, y_next, sol_next, tracker)

# ==========================================
# 4. MAIN ENTRY
# ==========================================
def solve_system_min_sum(buttons, joltage):
    num_vars = len(buttons)
    num_eqs = len(joltage)
    
    A = np.zeros((num_eqs, num_vars), dtype=int)
    for col, btn_indices in enumerate(buttons):
        for row in btn_indices:
            A[row, col] = 1
            
    y = np.array(joltage, dtype=int)
    
    # Initialize the Tracker
    tracker = {
        'min_sum': float('inf'), # Start with infinity
        'best_sol': None
    }
    
    recursive_solver_min_sum(A, y, [None] * num_vars, tracker)
    
    return tracker['best_sol']

In [23]:
start_time = time.perf_counter()

results = []
lights, buttons, joltages = parse_instructions(test)

for n, (i, _, k) in enumerate(zip(buttons, lights, joltages)):

    result = solve_system_min_sum(i, k)
    results.append(sum(result))

end_time = time.perf_counter()

elapsed = end_time - start_time
print(elapsed, "seconds")

0.0017598345875740051 seconds


In [24]:
assert sum(results) == 33

In [25]:
start_time = time.perf_counter()

results = []
lights, buttons, joltages = parse_instructions(data)

for n, (i, _, k) in enumerate(zip(buttons, lights, joltages)):

    result = solve_system_min_sum(i, k)
    results.append(sum(result))

end_time = time.perf_counter()

elapsed = end_time - start_time
print(elapsed, "seconds")

96.4559018323198 seconds


In [26]:
sum(results)

16978